<a href="https://colab.research.google.com/github/1212-site/Project/blob/main/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime
from datetime import date, timedelta
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
import warnings
plt.style.use('fivethirtyeight')

In [ ]:
from google.colab import files
up = files.upload()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
df = pd.read_excel('daily.xlsx', sheet_name=0)
# df = df.drop(0, axis=0)
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
df['Tanggal'] = df['Tanggal'].dt.normalize()

df['Tanggal'] = df['Tanggal'].dt.floor('D')
df.head()

In [ ]:
import plotly.express as px
figure = px.line(df, x="Tanggal", y="Pendapatan", title="Penjualan per-hari")
figure.show()

In [ ]:
result = seasonal_decompose(df["Pendapatan"], model='multiplicative', period=180)
fig = plt.figure()
fig = result.plot()
fig.set_size_inches(15,10)

In [ ]:
pd.plotting.autocorrelation_plot(df['Pendapatan'].loc[0:364])

In [ ]:
p = 5
d = 1

In [ ]:
plot_pacf(df["Pendapatan"], lags=30)

In [ ]:
q = 2

In [ ]:
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

model = sm.tsa.statespace.SARIMAX(df['Pendapatan'], order=(p,d,q),seasonal_order=(p,d,q,12))

modeled = model.fit()
print(modeled.summary())

In [ ]:
pred = modeled.predict(len(df), len(df)+364)

In [ ]:
pred

In [ ]:
pred = list(pred)

In [ ]:
testing = pd.DataFrame(pred, columns=['Hasil'])
testing

In [ ]:
month = []
for i in range(1,365):
  m = i
  month.append(m)

In [ ]:
test_am = pd.DataFrame(month, columns=['Hari-ke'])
test_am['Prediksi'] = testing['Hasil']

In [ ]:
pd.options.display.float_format = '{:.0f}'.format
test_am

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
ax.yaxis.get_major_formatter().set_scientific(False)
ax.yaxis.get_major_formatter().set_useOffset(False)

x = month
y = df['Pendapatan'].loc[0:363]
plt.plot(x,y)

x1 = month
y1 = df['Pendapatan'].loc[364:727]
plt.plot(x1,y1)

x2 = month
y2 = test_am['Prediksi']
plt.plot(x2,y2)

plt.xlabel("Hari Ke-")
plt.ylabel("Pendapatan")
plt.title('Perbandingan Pendapatan - Prediksi Monte Carlo')
plt.legend(['2022','2023','2024'])
plt.show()

In [ ]:
pp2023 = df['Pendapatan'].loc[0:363].reset_index(drop=True)
pp2022 = df['Pendapatan'].loc[364:727].reset_index(drop=True)

In [ ]:
data = pd.DataFrame(df['Tanggal'].loc[364:727])
data.reset_index(drop=True, inplace=True)
data['Pendapatan_2022'] = pp2022
data['Pendapatan_2023'] = pp2023
data['Pendapatan_2024'] = test_am['Prediksi']
data

In [ ]:
plt.ticklabel_format(style='plain')
df['Pendapatan'].plot(legend=True, label="Training Data", figsize=(15,10))
pred.plot(legend=True, label="Prediksi")

In [ ]:
dfnew = pd.DataFrame(df)
dfnew

In [ ]:
x = np.arange(1,730)
x_ttl = x.sum()
y = dfnew['Pendapatan']

In [ ]:
xyi = (x * y)
xy = []
for i in range(0,729):
  tmp = int(xyi[i])
  xy.append(tmp)

In [ ]:
import math
xq = x ** 2
y_ttl = y.sum()
y_avg = y.mean()
y_avg = math.ceil(y_avg * 100)/100.0
x_avg = x.mean()
xy_ttl = np.array(xy).sum()
xq_ttl = xq.sum()
xq_avg = xq.mean()
xq_avg = math.ceil(xq_avg * 100)/100.0
n = len(y)

In [ ]:
rt_y = []
for i in range(0,364):
  tmp = (y[i]+y[i+364])/2
  rt_y.append(tmp)

In [ ]:
test_arim = pd.DataFrame(month, columns=['Hari-ke'])
test_arim['Rata-Rata Penjualan'] = rt_y
test_arim['Prediksi'] = test_am['Prediksi']
test_arim

In [ ]:
test_arim.isnull().sum()

In [ ]:
mae_am = []
for i in range(0,364):
  tmp = abs((rt_y[i]-test_am['Prediksi'].loc[i]))
  mae_am.append(tmp)
test_am['MAE'] = mae_am
test_am

In [ ]:
n = 730

In [ ]:
maesum = test_am['MAE'].sum()
mcsum = test_am['Prediksi'].sum()
mape = 1/n*(maesum/mcsum)*100

In [ ]:
print('Total Prediksi : ', mcsum)
print('Total MAE : ', maesum)
print('MAPE : ', mape)